In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip -q install spacy
!python -m spacy download en
!python -m spacy download en_core_web_sm
!pip install -U spacy
!python -m spacy info

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 9.7 MB/s 
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 2.2.5
    Uninstalling en-core-web-sm-2.2.5:
      Successfully uninstalled en-core-web-sm-2.2.5
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.3.0/en_core_web_sm-3.3.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public

In [5]:
!python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 33.5 MB 2.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


## ДЗ 5 Тема “POS-tagger и NER”
#### Задание 1. Написать теггер на данных с русским языком
проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации

написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов

сравнить все реализованные методы сделать выводы


In [9]:
import nltk
from nltk.tokenize import word_tokenize
import matplotlib
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger
import pyconll
from corus import load_ne5

import spacy
from spacy import displacy
import en_core_web_md


import warnings
warnings.filterwarnings("ignore")

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [10]:
!wget -O ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
!wget -O ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-06-12 19:33:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-b.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42819806 (41M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train.conllu’

ru_syntagrus-ud-tra 100%[===================>]  40.84M  --.-KB/s    in 0.1s    

2022-06-12 19:33:41 (321 MB/s) - ‘ru_syntagrus-ud-train.conllu’ saved [42819806/42819806]

--2022-06-12 19:33:41--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108

In [11]:
train_data = pyconll.load_from_file('ru_syntagrus-ud-train.conllu')
test_data = pyconll.load_from_file('ru_syntagrus-ud-dev.conllu')

In [12]:
fdata_train = []
for sent in train_data[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in test_data[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in test_data[:]:
    fdata_sent_test.append([token.form for token in sent])

In [13]:
default_tagger = nltk.DefaultTagger('NN')
display(default_tagger.evaluate(fdata_test))

0.0

In [14]:
unigram_tagger = UnigramTagger(fdata_train)
display(unigram_tagger.evaluate(fdata_test))

0.8317338368383358

In [15]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
display(bigram_tagger.evaluate(fdata_test))

0.8379842437658702

In [16]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
display(trigram_tagger.evaluate(fdata_test))

0.8367016081776157

In [17]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.836792759945309

In [18]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append( '' if tok[1] is None else tok[1])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [19]:
le = LabelEncoder()

train_enc_labels = le.fit_transform(train_label)

test_enc_labels = le.transform(test_label)

In [20]:
countvec = CountVectorizer(ngram_range=(1, 1))

X_train = countvec.fit_transform(train_tok)
X_test = countvec.transform(test_tok)

lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

accuracy_score(test_enc_labels, pred)

0.23593983983332248

In [21]:
hashvec = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=50)

X_train = hashvec.fit_transform(train_tok)
X_test = hashvec.transform(test_tok)

lr = LogisticRegression(random_state=0, max_iter=10)
lr.fit(X_train, train_enc_labels)

pred = lr.predict(X_test)

accuracy_score(test_enc_labels, pred)

0.34734032163552314

#### Задание 2. Проверить насколько хорошо работает NER
данные брать из http://www.labinform.ru/pub/named_entities/

проверить NER из nltk/spacy/deeppavlov

написать свой нер попробовать разные подходы

передаём в сетку токен и его соседей

передаём в сетку только токен

свой вариант

сравнить ваши реализованные подходы на качество (вывести precision/recall/f1_score)


In [22]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-06-12 19:34:39--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip.1’

collection5.zip.1   100%[===================>]   1.81M  6.34MB/s    in 0.3s    

2022-06-12 19:34:40 (6.34 MB/s) - ‘collection5.zip.1’ saved [1899530/1899530]



In [23]:
#!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [24]:
path = 'Collection5/'
records = load_ne5(path)
document = next(records).text

In [26]:
ner_nltk = {(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

In [27]:
nltk.pos_tag(nltk.word_tokenize(document))

[('Обаму', 'JJ'),
 ('и', 'NNP'),
 ('Саркози', 'NNP'),
 ('выдвинули', 'NNP'),
 ('на', 'NNP'),
 ('Нобелевскую', 'NNP'),
 ('премию', 'NNP'),
 ('мира', 'NNP'),
 ('В', 'NNP'),
 ('список', 'NNP'),
 ('лиц', 'NNP'),
 ('и', 'NNP'),
 ('организаций', 'NNP'),
 (',', ','),
 ('выдвинутых', 'NNP'),
 ('на', 'NNP'),
 ('получение', 'NNP'),
 ('Нобелевской', 'NNP'),
 ('премии', 'NNP'),
 ('мира', 'NNP'),
 ('в', 'NNP'),
 ('2009', 'CD'),
 ('году', 'NNP'),
 (',', ','),
 ('попали', 'NNP'),
 ('президент', 'NNP'),
 ('США', 'NNP'),
 ('Барак', 'NNP'),
 ('Обама', 'NNP'),
 ('и', 'NNP'),
 ('президент', 'NNP'),
 ('Франции', 'NNP'),
 ('Николя', 'NNP'),
 ('Саркози', 'NNP'),
 ('.', '.'),
 ('Об', 'VB'),
 ('этом', 'JJ'),
 ('сообщает', 'NNP'),
 ('AFP', 'NNP'),
 ('.', '.'),
 ('По', 'VB'),
 ('официальным', 'JJ'),
 ('данным', 'NNP'),
 (',', ','),
 ('в', 'NNP'),
 ('список', 'NNP'),
 ('включены', 'VBD'),
 ('205', 'CD'),
 ('номинантов', 'NNP'),
 (',', ','),
 ('33', 'CD'),
 ('из', 'NN'),
 ('которых', 'NNP'),
 ('являются', 'NNP'),


In [30]:
nlp = en_core_web_md.load()
doc = nlp(document)

In [42]:
displacy.render(doc, jupyter=True, style='ent')